# Penndulum Model Codes

## Dependencies

In [1]:
import os
import shutil
import random; random.seed(42)

import pandas as pd
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import torch
from torch import nn
from reservoirpy import mat_gen, ESN

## Directories:

In [2]:
# training data
train_dir = '/raid/cs152/zxaa2018/penndulum/train_and_test_split/dpc_dataset_traintest_4_200_csv/train'
train_dir_video = '/raid/cs152/zxaa2018/penndulum//train_and_test_split/dpc_dataset_traintest_4_200_h264/train'

# test data
test_inputs_dir = '/raid/cs152/zxaa2018/penndulum/train_and_test_split/dpc_dataset_traintest_4_200_csv/test_inputs/'
test_targets_dir = '/raid/cs152/zxaa2018/penndulum/train_and_test_split/dpc_dataset_traintest_4_200_csv/test_targets/'
test_targets_video = '/raid/cs152/zxaa2018/penndulum/train_and_test_split/dpc_dataset_traintest_4_200_h264/test_targets/'

# validation data
validation_inputs_dir = '/raid/cs152/zxaa2018/penndulum/train_and_test_split/dpc_dataset_traintest_4_200_csv/validation_inputs/'
validation_targets_dir = '/raid/cs152/zxaa2018/penndulum/train_and_test_split/dpc_dataset_traintest_4_200_csv/validation_targets/'
validation_targets_video = '/raid/cs152/zxaa2018/penndulum/train_and_test_split/dpc_dataset_traintest_4_200_h264/validation_targets/'

## Functions and Class Definitions

### Data Transformation Functions

In [3]:
# some constants
DEFAULT_X_RED, DEFAULT_Y_RED = (240, 232)

PIXEL_DISTANCE_GREEN_TO_RED = 118 # approx. value | calculated with the Pythagorean theorem and averaged: np.sqrt((y_green-y_red)**2 + (x_green-x_red)**2)
PIXEL_DISTANCE_BLUE_TO_GREEN = 90 # approx. value | calculated with the Pythagorean theorem and averaged: np.sqrt((y_blue-y_green)**2 + (x_blue-x_green)**2)

def raw_to_pixel(l):
    '''Convert the raw coordinates to pixel coordinates.'''
    assert isinstance(l, list)
    return [x/5 for x in l]


def pixel_to_raw(l):
    '''Convert the pixel coordinates to raw coordinates.'''
    assert isinstance(l, list)
    return [x*5 for x in l]


def raw_cartesian_to_polar_angles(l):
    '''Convert the cartesian coordinates to polar coordinates.'''
    assert isinstance(l, list)
    x_red, y_red, x_green, y_green, x_blue, y_blue = raw_to_pixel(l)

    angle_green_red = np.arctan((y_green-y_red)/(x_green-x_red+0.001))
    angle_blue_green = np.arctan((y_blue-y_green)/(x_blue-x_green+0.001))
    
    return [np.sin(angle_green_red), np.cos(angle_green_red), np.sin(angle_blue_green), np.cos(angle_blue_green)]

def polar_angles_to_raw_cartesian(l):
    '''Convert the polar coordinates back to cartesian coordinates.'''
    assert isinstance(l, list)
    sin_angle_green_red, cos_angle_green_red, sin_angle_blue_green, cos_angle_blue_green = l
    
    y_green = PIXEL_DISTANCE_GREEN_TO_RED * sin_angle_green_red + DEFAULT_Y_RED
    x_green = PIXEL_DISTANCE_GREEN_TO_RED * cos_angle_green_red + DEFAULT_X_RED

    y_blue = PIXEL_DISTANCE_BLUE_TO_GREEN * sin_angle_blue_green + y_green
    x_blue = PIXEL_DISTANCE_BLUE_TO_GREEN * cos_angle_blue_green + x_green
    
    return pixel_to_raw([DEFAULT_X_RED, DEFAULT_Y_RED, x_green, y_green, x_blue, y_blue])

### Data reading functions

Parsing training data:
training data x-y matching is like this:
x: a list of 4 frames
y: the frame that follows

In [4]:
def parse_csv_esn(csv_file,seq_len = 0):
    '''Given a csv file and a length of sequence, return the sequence in list of list'''
    disabled = seq_len == 0
    X_data = []
    f = pd.read_csv(csv_file, header=None, delim_whitespace=True, engine='python')
    temp = []
    for i, row in f.iterrows():
        if (not disabled) and i>= seq_len:
            break
        next_frame = raw_cartesian_to_polar_angles(row.to_list())
        X_data.append(next_frame.copy())
        
    return X_data

In [5]:
def get_seq_list(source_dir,seq_len = 0):
    result = []
    for filename in tqdm([x for x in os.listdir(source_dir) if not x.startswith('.')]):
        # load in a file
        X_data = parse_csv_esn(os.path.join(source_dir, filename),500)
        result.append(X_data)
    return result

### Dataset Definition

In [6]:
class DoublePendulumDatasetESN():
    def __init__(self,X_list):
        self.sampleList = X_list
    
    def __len__(self):
        return len(self.sample_list)

### Model Definition

Data Visualization Functions

In [7]:
def plot_trajectory_from_tensor(coords: torch.Tensor):
    blue_x = coords.index_select(1,torch.tensor([4])).numpy()
    blue_y = coords.index_select(1,torch.tensor([5])).numpy()
    plt.scatter(blue_x,blue_y)

## Training, testing and analysis Codes

### Loading Training Data

In [8]:
BATCH_SIZE = 1000

# load in all separate files
Seq_train = get_seq_list(train_dir,seq_len=500)

Seq_valid_input = get_seq_list(validation_inputs_dir)

Seq_valid_target = get_seq_list(validation_targets_dir)


100%|██████████| 24/24 [00:00<00:00, 29.17it/s]


40

In [43]:
train_X = [np.array(x[:250]) for x in Seq_train]
train_y = [np.array(x[250:]) for x in Seq_train]

In [41]:
train_y[0].shape

(400, 4)

### Hyperparameter and Model instantiate

In [15]:
units = 500
leak_rate = 0.2
spectral_radius = 1.0
input_scaling = 1.
density = 0.1
input_connectivity = 1.0
regularization = 1e-6
seed = 1234

Win = mat_gen.generate_input_weights(units, 4, input_scaling=input_scaling,
                                     proba=input_connectivity, input_bias=True,
                                     seed=seed)

W = mat_gen.generate_internal_weights(units, sr=spectral_radius,
                              proba=density, seed=seed)

reservoir = ESN(leak_rate, W, Win, ridge=regularization)

### Training

In [44]:
states = reservoir.train(train_X, train_y, verbose=True, return_states=True)

PermissionError: [Errno 13] Permission denied: '/tmp/reservoirpy-temp/RidgeRegression7600de99-578b-4dc2-b73d-41572a55ae25.dat'

In [88]:
generation = esn.generate(n=len(Seq_valid_input[0]), inputData=None, initialOutputData=Seq_valid_input[0][0])

### Testing Data Loading